In [1]:
import csv
with open('/van_gogh_paintings_description.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
data = data[1:]

In [2]:
import boto3
import csv
from tqdm import tqdm
from pydub import AudioSegment
def split_text(text, max_length):
    # This function splits the text into chunks of max_length, splitting at sentence ends if possible
    sentences = text.split('. ')
    chunks = []
    current_chunk = ''

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length:
            current_chunk += sentence + '. '
        else:
            chunks.append(current_chunk)
            current_chunk = sentence + '. '
    
    if current_chunk:
        chunks.append(current_chunk)

    return chunks
polly_client = boto3.client('polly',
                            aws_access_key_id='',
                            aws_secret_access_key='',
                            region_name='us-east-1')
s3_client = boto3.client('s3',
                         aws_access_key_id='',
                         aws_secret_access_key='',
                         region_name='us-east-1')
for i in tqdm(range(len(data))):
    if data[i][-2] == '':
        continue
    long_text = data[i][-2]
    chunks = split_text(long_text, 3000)
    temp_files = []

    for _, chunk in enumerate(chunks):
        response = polly_client.synthesize_speech(VoiceId='Joanna',
                                                OutputFormat='mp3', 
                                                Text=chunk)
        
        temp_file = f"temp_{i}.mp3"
        with open(temp_file, 'wb') as file:
            file.write(response['AudioStream'].read())
        temp_files.append(temp_file)

    # Concatenate audio files
    combined = AudioSegment.empty()
    for temp_file in temp_files:
        segment = AudioSegment.from_mp3(temp_file)
    combined += segment
    
    combined_file = data[i][0] + '.mp3'
    combined.export(combined_file, format="mp3")
    bucket_name = 'artaudio'
    with open(combined_file, 'rb') as file:
        s3_client.upload_fileobj(file, bucket_name, combined_file)


100%|██████████| 153/153 [02:43<00:00,  1.07s/it]


In [3]:
i

152